In [ ]:
import pandas as pd
import folium
import plotly.graph_objects as go
import plotly.express as px
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Standorte

In [ ]:
m = folium.Map(location=[51.2099, 10.4571], zoom_start=6)
folium.Marker([49.2359910863105, 6.993367970054943], tooltip="Bahnhofstraße (Mitte), Saarbrücken").add_to(m)
folium.Marker([52.374956125557894, 9.737387597107245], tooltip="Georgstraße, Hannover").add_to(m)
folium.Marker([53.55376220607088, 9.991887654803696], tooltip="Jungfernstieg, Hamburg").add_to(m)
folium.Marker([48.137685457750635, 11.573367770032743], tooltip="Kaufingerstraße, München").add_to(m)
folium.Marker([54.088503508553394, 12.134802012486357], tooltip="Kröpeliner Straße (West), Rostock").add_to(m)
folium.Marker([551.222185050102674, 6.779255270096054], tooltip="Königsallee Ostseite (Süd), Düsseldorf").add_to(m)
folium.Marker([48.77820910005942, 9.177711579841167], tooltip="Königstraße (Mitte), Stuttgart").add_to(m)
folium.Marker([53.07736868080147, 8.804847969315563], tooltip="Obernstraße, Bremen").add_to(m)
folium.Marker([51.047494158917196, 13.737369656599013], tooltip="Prager Straße (Nord), Dresden").add_to(m)
folium.Marker([50.001027164285354, 8.270445585412894], tooltip="Stadthausstraße, Mainz").add_to(m)
folium.Marker([50.11494292444826, 8.687410770070198], tooltip="Zeil (Mitte), Frankfurt a. M.").add_to(m)

m

# Daten einlesen

Um Daten einzulesen, müssen wir eine Verbindung mit der MongoDB erstellen.

In [ ]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["HyStreet"]
hystreet = db["HyStreetData"]

cursor = hystreet.find()
data = pd.DataFrame(list(cursor))

In [ ]:
addresses = hystreet.distinct("address")
weatherConditions = hystreet.distinct("weatherCondition")
temps = hystreet.distinct("temperature")

hystreet.distinct("city")

# Hilfsfunktionen definieren

In [ ]:
def sumOfYear(data, year):
    result = []
    
    for address in addresses:
        filteredByYear = data.loc[data['address'] == address].loc[data['year'] == year]
        result.append((address, filteredByYear["pedestrians"].sum()))
        
    return result

# sumOfYear(data, 2021)

In [ ]:
def sumOfYearPerMonth(data, year):
    result = []
    month = [
        ["January",1],
        ["February",2],
        ["March",3],
        ["April",4],
        ["May",5],
        ["June",6],
        ["July",7],
        ["August",8],
        ["September",9],
        ["October",10],
        ["November",11],
        ["December",12]
    ]

    for address in addresses:
        filteredByYear = data.loc[data['address'] == address].loc[data['year'] == year]
        for i in month:
            filteredByMonth = filteredByYear.loc[filteredByYear['month'] == i[1]]
            result.append((address, str(i[0]) + " " + str(year), filteredByMonth['pedestrians'].sum()))

    return result

# sumOfYearPerMonth(data, 2022)

In [ ]:
def sumOfYearPerWeekday(data, year):
    result = []
    weekdays = [
        'Monday',
        'Tuesday',
        'Wednesday',
        'Thursday',
        'Friday',
        'Saturday',
        'Sunday'
    ]
    
    for address in addresses:
        filteredByYear = data.loc[data['address'] == address].loc[data['year'] == year]
        for day in weekdays:
            filteredByWeekday = filteredByYear.loc[filteredByYear['weekday'] == day]
            result.append((address, day, filteredByWeekday['pedestrians'].sum()))

    return result

# sumOfYearPerWeekday(data, 2021)

In [ ]:
def sumOfYearPerDay(data, year):
    result = []

    for address in addresses:
        filteredByYear = data.loc[data['address'] == address].loc[data['year'] == year]
        for i in range(1,13):
            filteredByMonth = filteredByYear.loc[filteredByYear['month'] == i]
            for d in range(1,32):
                pedestriansPerDay = filteredByMonth.loc[filteredByMonth['day'] == d]
                if not pedestriansPerDay.empty:
                    result.append((address, i, d, pedestriansPerDay['pedestrians'].sum()))

    return result

# sumOfYearPerDay(data, 2020)

In [ ]:
sumOverAllYear = sumOfYear(data, 2022) + sumOfYear(data, 2021) + sumOfYear(data, 2020)
long_df = pd.DataFrame(sumOverAllYear, columns=['location', 'countp'])

sumPersonOverAllYears = 0
anzahlNuller = 0

for persons in range(0,len(long_df.index)):
    if long_df['countp'][persons] != 0:
        sumPersonOverAllYears = sumPersonOverAllYears + long_df['countp'][persons]
    else:
        anzahlNuller = anzahlNuller + 1

# durch anzahl der Zonen / Durch anzahl der Jahre / Durch anzahl der Jahre 
sumPersonForAZoneAYear = sumPersonOverAllYears / len(addresses) / 3
sumPersonforAMonth = sumPersonForAZoneAYear / 12
sumPersonforADay = sumPersonforAMonth / 30.44
# sumPersonsforADay

In [ ]:
def getWeatherConditionRawData():
    result = []
    for weatherCondition in weatherConditions:
        filteredByWeatherCondition = data.loc[data['weatherCondition'] == weatherCondition]
        result.append((weatherCondition, filteredByWeatherCondition["pedestrians"].sum(), len(filteredByWeatherCondition.index)))

    result_cleaned = []
    clearPed = 0
    clearCnt = 0
    partCloudPed = 0
    partCloudCnt = 0
    for i in result:
        if i[0] == 'clear-day':
            clearPed = clearPed + i[1]
            clearCnt = clearCnt + i[2]
        elif i[0] == 'clear-night':
            clearPed = clearPed + i[1]
            clearCnt = clearCnt + i[2]
        elif i[0] == 'partly-cloudy-day':
            partCloudPed = partCloudPed + i[1]
            partCloudCnt = partCloudCnt + i[2]
        elif i[0] == 'partly-cloudy-night':
            partCloudPed = partCloudPed + i[1]
            partCloudCnt = partCloudCnt + i[2]
        else:
            result_cleaned.append(i)
    clearPedTuple = ('clear', clearPed, clearCnt)
    result_cleaned.append(clearPedTuple)

    partCloudPedTuple = ('partly-cloudy', partCloudPed, partCloudCnt)
    result_cleaned.append(partCloudPedTuple)
    
    return result_cleaned

# getWeatherConditionRawData()

In [ ]:
def getTempPed():
    result = []
    for temp in temps:
        filteredByTemp = data.loc[data['temperature'] == temp]
        result.append((round(int(temp),0), filteredByTemp["pedestrians"].sum(), len(filteredByTemp.index)))
    
    return result

# getTempPed()

In [ ]:
def getTempPerWeatherCondition():
    result = []
    for weatherCondition in weatherConditions:
        filteredByWeatherCondition = data.loc[data['weatherCondition'] == weatherCondition]
        result.append((weatherCondition, round(filteredByWeatherCondition["temperature"].sum()/len(filteredByWeatherCondition.index), 2)))

    result_cleaned = []
    avgTempClear = 0
    avgTempPartCloud = 0
    for i in result:
        if i[0] == 'clear-day':
            avgTempClear = avgTempClear + i[1]
        elif i[0] == 'clear-night':
            avgTempClear = avgTempClear + i[1]
        elif i[0] == 'partly-cloudy-day':
            avgTempPartCloud = avgTempPartCloud + i[1]
        elif i[0] == 'partly-cloudy-night':
            avgTempPartCloud = avgTempPartCloud + i[1]
        else:
            result_cleaned.append(i)
    clearTuple = ('clear', avgTempClear)
    result_cleaned.append(clearTuple)

    partCloudTuple = ('partly-cloudy', avgTempPartCloud)
    result_cleaned.append(partCloudTuple)
    
    return result_cleaned

# getTempPerWeatherCondition()

# Daten visualisieren

## Jahresvergleich

### Fußgänger pro Fußgängerzone im Jahr 2020

In [ ]:
absPed2020 = sumOfYear(data, 2020)
fig0 = px.bar(absPed2020, x=[x[0] for x in absPed2020], y=[y[1] for y in absPed2020], title='Fußgänger pro Fußgängerzone im Jahr 2020', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'}).update_xaxes(categoryorder="total descending")
fig0.show()

### Fußgänger pro Fußgängerzone im Jahr 2021

In [ ]:
absPed2021 = sumOfYear(data, 2021)
fig1 = px.bar(absPed2021, x=[x[0] for x in absPed2021], y=[y[1] for y in absPed2021], title='Fußgänger pro Fußgängerzone im Jahr 2021', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'}).update_xaxes(categoryorder="total descending")
fig1.show()

### Fußgänger pro Fußgängerzone im Jahr 2022

In [ ]:
absPed2022 = sumOfYear(data, 2022)
fig2 = px.bar(absPed2022, x=[x[0] for x in absPed2022], y=[y[1] for y in absPed2022], title='Fußgänger pro Fußgängerzone im Jahr 2022', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'}).update_xaxes(categoryorder="total descending")
fig2.show()

### Vergleich der drei Jahre

In [ ]:
comparisonYear = pd.DataFrame(columns=['Ort', 'Jahr', 'Passanten'])

for i in range(len(absPed2020)):
    comparisonYear = comparisonYear.append({'Ort' : absPed2020[i][0], 'Jahr' : 2020, 'Passanten' : absPed2020[i][1] }, ignore_index=True)
    comparisonYear = comparisonYear.append({'Ort' : absPed2020[i][0], 'Jahr' : 2021, 'Passanten' : absPed2021[i][1] }, ignore_index=True)

for i in range(len(absPed2022)):
    comparisonYear = comparisonYear.append({'Ort' : absPed2022[i][0], 'Jahr' : 2022, 'Passanten' : absPed2022[i][1] }, ignore_index=True)

fig3 = px.histogram(comparisonYear, x='Ort', y='Passanten', color='Jahr', barmode='group', title='Vergleich 2020 / 2021 / 2022 (2022 nur bis November)', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'}).update_xaxes(categoryorder="total descending")
fig3.show()

Man sieht, dass die Corona-Jahre die Fußgängeranzahl in den Städten deutlich reduziert haben. So sind bereits im Jahr 2022 **bis zum Monat November** mehr Fußgänger unterwegs gewesen als in den beiden Jahren zuvor.

### Anzahl Fußgänger pro Wochentag von 2020 bis Herbst 2022

In [ ]:
weeklyPed2022 = sumOfYearPerWeekday(data, 2022)

long_df = pd.DataFrame(weeklyPed2022, columns=['Fußgängerzone', 'Wochentag', 'Anzahl Fußgänger von 2020 bis Herbst 2022'])
# print(long_df)
fig4 = px.bar(long_df, x='Wochentag', y='Anzahl Fußgänger von 2020 bis Herbst 2022', color='Fußgängerzone', title='Anzahl Fußgänger pro Wochentag von 2020 bis Herbst 2022')
fig4.show()

## Vergleich der Relation zwischen Einwohneranzahl und Passanten pro Jahr
(in Welcher Stadt wird viel gelaufen)

In [ ]:
# Datenquelle: Wikipedia
inhabitants = [
    ('Bahnhofstraße (Mitte), Saarbrücken', 178151),
    ('Georgstraße, Hannover', 532163),
    ('Jungfernstieg, Hamburg', 1853935),
    ('Kaufingerstraße, München', 1487708),
    ('Kröpeliner Straße (West), Rostock', 208400),
    ('Königsallee Ostseite (Süd), Düsseldorf', 619477),
    ('Königstraße (Mitte), Stuttgart', 626275),
    ('Obernstraße, Bremen', 563290),
    ('Prager Straße (Nord), Dresden', 555351),
    ('Stadthausstraße, Mainz', 217556),
    ('Zeil (Mitte), Frankfurt a.M.', 759224),
]

result = []
for address in addresses:
    city = data.loc[data['address'] == address]
    for i in inhabitants:
        cityInhabitants = 0
        if i[0] == address:
            cityInhabitants = i[1]
            break
    result.append((address, round(city['pedestrians'].sum()/len(city.index), 2), cityInhabitants))

inhabitants_long_df = pd.DataFrame(result, columns=['Stadt', 'Stündliche Fußgänger', 'Einwohnerzahl'])

fig = px.scatter(inhabitants_long_df, x="Einwohnerzahl", y="Stündliche Fußgänger", color="Stadt", title='Stündliche Fußgänger im Verhältnis zur Einwohnerzahl')
fig.show()

## Jahreszeiten und Monatsvergleich Vergleich

Im Folgenden werden die Jahreszeiten zunächst anhand derer absoluten Zahlen verglichen. Dafür wird das Jahr in die folgenden Abschnitte unterteilt: *Winter* (Dezember, Januar, Februar), *Frühling* (März, April, Mai), *Sommer* (Juni, Juli, August) und *Herbst* (September, Oktober, November). Hier werden die **meteorologischen Jahrezeiten** verwendet.

In [ ]:
seasons = list()
for city in hystreet.distinct("address"):
    seasons.append(data.loc[data["address"] == city])

def getRowsOfPeriod(sYear, sMonth, sDay, eYear, eMonth, eDay, data):
    '''Returns a dataframe with all the rows between the given dates (The given dates are inclusive).'''
    df = data.copy()
    for i, city in enumerate(df):
        city = city.loc[(city.year >= sYear) & (city.year <= eYear)]
        
        city = city.drop( city[(city.year == sYear) & (city.month < sMonth) ].index)
        city = city.drop( city[(city.year == eYear) & (city.month > eMonth) ].index)
    
        city = city.drop( city[(city.year == sYear) & (city.month == sMonth) & (city.day < sDay)].index)
        city = city.drop( city[(city.year == eYear) & (city.month == eMonth) & (city.day > eDay)].index)
    
        df[i] = city
    return df


def sumOfDf(dataframe):
    '''Returns the sum of the pedestrian column for a given dataframe.'''
    if not dataframe.empty:
        return dataframe['pedestrians'].sum()
    return 0


def sumOfDfs(dfList):
    '''Returns the sum of all the pedestrian sums for the given dataframes in the list'''
    result = 0
    for df in dfList:
        result += sumOfDf(df)
    return result
    

def seasonsOfYear(year, data):
    winterP1 = sumOfDfs(getRowsOfPeriod(year, 1, 1, year, 2, 30, data))
    winter = sumOfDfs(getRowsOfPeriod(year, 12, 1, year, 12, 31, data)) + winterP1
    spring = sumOfDfs(getRowsOfPeriod(year, 3, 1, year, 5, 31, data))
    sommer = sumOfDfs(getRowsOfPeriod(year, 6, 1, year, 8, 31, data))
    autumn = sumOfDfs(getRowsOfPeriod(year, 9, 1, year, 11, 31, data))
    
    result = [
        ('winter', winter),
        ('spring', spring),
        ('sommer', sommer),
        ('autumn', autumn)
    ]
    
    return pd.DataFrame(result, columns=['season', 'count'])

Betrachten wir nun die die Jahreszeiten der beiden Jahre 2020 und 2021

In [ ]:
absSeasons2020 = seasonsOfYear(2020, seasons)
absSeasons2021 = seasonsOfYear(2021, seasons)

comparisonSeasons = pd.DataFrame(columns=['Jahreszeit', 'Jahr', 'Passanten'])
for i in range(len(absSeasons2020)):
    comparisonSeasons = comparisonSeasons.append({'Jahreszeit' : absSeasons2020.iloc[i][0], 'Jahr' : 2020, 'Passanten' : absSeasons2020.iloc[i][1] }, ignore_index=True)
    comparisonSeasons = comparisonSeasons.append({'Jahreszeit' : absSeasons2020.iloc[i][0], 'Jahr' : 2021, 'Passanten' : absSeasons2021.iloc[i][1] }, ignore_index=True)

    
figSeasons = px.histogram(comparisonSeasons, x='Jahreszeit', y='Passanten', color='Jahr', barmode='group', title='Vergleich Jahreszeiten 2020 / 2021').update_xaxes(categoryorder="total descending")
figSeasons.show()

Die Jahreszeiten können zusätzlich noch nach den einzelnen Monaten aufgeschlüsselt werden um einen genaueren Einblick zu schaffen. Auch hier werden die beiden Jahre 2020 und 2021 verglichen.

In [ ]:
def monthsOfYear(year, data):
    '''Returns a dataframe containing all months in combination with the sum of all locations for that specific month'''
    df = data.copy()
    for idx, city in enumerate(df):
        df[idx] = city.loc[city.year == year]
    
    months = [0 for idx in range(12)]
    sMonth = ["Januar", "Februar", "März", "April", "Mai", "Juni", "Juli", "August", "September", "Oktober", "November", "Dezember"]
    
    for city in df:
        for month in range(1, 13):
            mDf = city.loc[city.month == month]
            months[month - 1] += sumOfDf(mDf)
            
    result = { 'Month' : sMonth, 'Count' : months }
            
    return pd.DataFrame(result)

Betrachten wir nun die Monate der Jahre 2020 und 2021 nebeneinander.

In [ ]:
absMonths2020 = monthsOfYear(2020, seasons)
absMonths2021 = monthsOfYear(2021, seasons)

comparisonMonths = pd.DataFrame(columns=['Monat', 'Jahr', 'Passanten'])
for i in range(len(absMonths2020)):
    comparisonMonths = comparisonMonths.append({'Monat' : absMonths2020.iloc[i][0], 'Jahr' : 2020, 'Passanten' : absMonths2020.iloc[i][1] }, ignore_index=True)
    comparisonMonths = comparisonMonths.append({'Monat' : absMonths2020.iloc[i][0], 'Jahr' : 2021, 'Passanten' : absMonths2021.iloc[i][1] }, ignore_index=True)

figMonths2021 = px.histogram(comparisonMonths, x='Monat', y='Passanten', color='Jahr', barmode='group', title='Vergleich Monate 2020 / 2021')
figMonths2021.show()

## Zu welcher Uhrzeit ist am meisten los?

In [ ]:
def absPerHour(data, years):
    dataTime = data.loc[data['year'].isin(years)]
    dataTL = list()
    monate = ["Januar", "Februar", "März", "April", "Mai", "Juni", "Juli", "August", "September", "Oktober", "November", "Dezember"]

    df = pd.DataFrame(columns=["Monat", "Stunde", "Passanten"])

    for month in range(12):
        tmp = dataTime.loc[dataTime['month'] == (month + 1)]
        tsum = tmp.groupby('hour')['pedestrians'].sum()

        for hour in range(0, 24):
            df = df.append({"Monat" : monate[month], "Stunde" : hour, "Passanten" : tsum.iloc[hour] }, ignore_index=True)
    
    return df
    
absHours = absPerHour(data, [2020])
figHours = px.line(absHours, x="Stunde", y="Passanten", color='Monat', title="Uhrzeitenvergleich der Monate des Jahres 2020")
figHours.show()

In [ ]:
absHours = absPerHour(data, [2021])
figHours = px.line(absHours, x="Stunde", y="Passanten", color='Monat', title="Uhrzeitenvergleich der Monate des Jahres 2021")
figHours.show()

In [ ]:
absHours = absPerHour(data, [2020, 2021])
figHours = px.line(absHours, x="Stunde", y="Passanten", color='Monat', title="Uhrzeitenvergleich der Monate des Jahres 2021")
figHours.show()

## Corona-Vergleich

### Monatliche Fußgänger von 2020 bis 2022
(deutlich zu erkennen: zwei Lockdowns)

In [ ]:
monthlyPed = sumOfYearPerMonth(data, 2020) + sumOfYearPerMonth(data, 2021) + sumOfYearPerMonth(data, 2022)
long_df = pd.DataFrame(monthlyPed, columns=['location', 'month', 'countp'])

fig = px.bar(long_df, x='month', y='countp', color="location", title='Monatliche Fußgänger von 2020 bis 2022')
fig.show()

### Wöchentliche Fußgänger von März 2020 bis Mai 2020
(deutlich zu erkennen: erster Lockdown)

In [ ]:
sumOf2020PerDay = sumOfYearPerDay(data, 2020)

marchToMayPed = [q for q in sumOf2020PerDay if q[1] == 3 or q[1] == 4 or q[1] == 5]
result = []

for address in addresses:
    filteredByAddress = [q for q in marchToMayPed if q[0] == address]

    i = 0
    sumPed = 0
    weekNumber = 1
    for day in filteredByAddress:
        sumPed = sumPed + day[3]
        
        if i == 6:
            result.append((address, weekNumber, sumPed))
            i = 0
            sumPed = 0
            weekNumber = weekNumber + 1
        else:
            i = i + 1

weekly2020MarchToMayPed_longdf = pd.DataFrame(result, columns=['Adresse', 'Wochennummer', 'Anzahl Fußgänger'])

fig = px.bar(weekly2020MarchToMayPed_longdf, x='Wochennummer', y='Anzahl Fußgänger', color='Adresse', title='Wöchentliche Fußgänger von März 2020 bis Mai 2020')
fig.show()

### Tägliche Fußgänger im März 2020 pro Stadt
(deutlich zu erkennen: Anfang des Lockdowns)

In [ ]:
sumOf2020PerDay = sumOfYearPerDay(data, 2020)
sumOf2020PerDay_long_df = pd.DataFrame(sumOf2020PerDay, columns=['Adresse', 'Monat', 'Tag', 'Anzahl Fußgänger'])
marchPed = sumOf2020PerDay_long_df[sumOf2020PerDay_long_df.Monat == 3]

fig = px.bar(marchPed, x='Tag', y='Anzahl Fußgänger', color='Adresse', title='Tägliche Fußgänger im März 2020 pro Stadt')
fig.show()

## Events

Wir wollen nun noch schauen, ob sich größere Events anhand der Anzahl der gezählten Passanten ausfindig machen lassen können. Wir betrachten dazu beispielhaft den Jungfernstieg in Hamburg. Analysiert werden hier die Freitage um jeweils 20 Uhr. Zunächst werden die Daten visualisiert.

In [ ]:
eData = data.loc[(data['city'] == ' Hamburg') & (data['year'] == 2021)  & (data['weekday'] == "Friday") & (data['hour'] == 20) ]
eData.reset_index(inplace=True)

figEvents = px.line(eData, x=eData.index, y="pedestrians", labels={"_index":"Freitage im Jahr 2021"}, title="Besucher pro Freitage um 20 Uhr im Jahr 2021")
figEvents.show()

In [ ]:
import numpy as np

Um automatisiert Events zu finden, bewegen wir ein Fenster individueller Größer über das gegebene Dataframe und bilden jeweils den Durchschnitt aus den gerade betrachteten Daten. Anschließend gehen wir die einzelnen Tage durch und schauen ob einer dieser Tage mind. 60% mehr Passanten hatte als der Durchschnitt. Sollte das der Fall sein, so speichern wir diesen Tag und geben die gesammelten Tage schließlich aus.

In [ ]:
def findEvents(data, width):
    eData = data.reset_index()
    result = list()
    for i in range(0, eData.shape[0] - width):
        subset = eData.iloc[i:i+width]
        mean = subset["pedestrians"].mean()

        for index, row in subset.iterrows():
            if row['pedestrians'] > mean * 1.6:
                result.append((row['level_0'], row['month'], row['day'], row['pedestrians'], row['weekday']))
    return set(result)

findEvents(eData, 4)

Wir sehen die 4 folgenden Ereignisse:
- 26.03.2021
- 28.05.2021
- 04.06.2021
- 26.11.2021

Die Freitage am 28.05 und 04.06 lassen sich mit einem erhöhten Temperaturanstieg erklären.
Der Freitag am 26.11.2021 lag in der Woche des Weihnachtsmarktes, somit wurde tatsächlich ein größeres Event gefunden.

## Wetter

### Durchschnittliche Fußgänger pro Stunde (in der das jeweilige Wetter vorherrschte)

In [ ]:
weatherConditionData = getWeatherConditionRawData()

result = []
for i in weatherConditionData:
    result.append((i[0], round(i[1]/i[2], 2)))


weatherConditions_long_df = pd.DataFrame(result, columns=['Wetter', 'Durchschnittliche Fußgänger pro Stunde'])

fig = px.bar(weatherConditions_long_df, x='Wetter', y='Durchschnittliche Fußgänger pro Stunde', title='Durchschnittliche Fußgänger pro Stunde (in der das jeweilige Wetter vorherrschte)').update_xaxes(categoryorder="total descending")
fig.show()

### Wetteranteile von 2020 bis Herbst 2022

In [ ]:
weatherConditionData = getWeatherConditionRawData()

sumHours = 0
for i in weatherConditionData:
    sumHours = sumHours + i[2]

result = []
for i in weatherConditionData:
    result.append((i[0], round(i[2]/sumHours,4)))

weatherConditions_long_df = pd.DataFrame(result, columns=['Wetter', 'Anteil am Wetter von 2020 bis Herbst 2022'])

fig = px.bar(weatherConditions_long_df, x='Wetter', y='Anteil am Wetter von 2020 bis Herbst 2022', title='Wetteranteile von 2020 bis Herbst 2022').update_xaxes(categoryorder="total descending")
fig.show()

### Durchschnittliche Fußgänger pro Stunde und Gradzahl von 2020 bis Herbst 2022

In [ ]:
result1 = getTempPed()

result = []
for i in result1:
    result.append((i[0], round(i[1]/i[2], 2)))

temp_long_df = pd.DataFrame(result, columns=['Temperatur', 'Durchschnittliche Fußgänger pro Stunde von 2020 bis Herbst 2022'])

fig = px.bar(temp_long_df, x='Temperatur', y='Durchschnittliche Fußgänger pro Stunde von 2020 bis Herbst 2022', title='Durchschnittliche Fußgänger pro Stunde und Gradzahl von 2020 bis Herbst 2022')
fig.show()

### Temperaturverteilung von 2020 bis Herbst 2022

In [ ]:
result1 = getTempPed()

sumHours = 0
for i in result1:
    sumHours = sumHours + i[2]

result = []
for i in result1:
    result.append((i[0], round(i[2]/sumHours, 5)))

temp_long_df = pd.DataFrame(result, columns=['Temperatur', 'Anteil Temperatur'])

fig = px.bar(temp_long_df, x='Temperatur', y='Anteil Temperatur', title='Temperaturverteilung von 2020 bis Herbst 2022')
fig.show()

### Durchschnittliche Temperatur pro Wetter von 2020 bis Herbst 2022

In [ ]:
result = getTempPerWeatherCondition()

temp_long_df = pd.DataFrame(result, columns=['Wetter', 'Temperatur'])

fig = px.bar(temp_long_df, x='Wetter', y='Temperatur', title='Durchschnittliche Temperatur pro Wetter von 2020 bis Herbst 2022').update_xaxes(categoryorder="total descending")
fig.show()